# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
# Read CSV
Vacation_Cities = pd.read_csv("../WeatherPy/output_data/cities.csv")
Vacation_Cities.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lata,40.16,-8.33,55.00,100,75,3.36,PT,1607530078
1,1,ushuaia,-54.80,-68.30,53.60,66,75,11.41,AR,1607529952
2,2,busselton,-33.65,115.33,63.00,84,0,1.01,AU,1607530078
3,3,soyo,-6.13,12.37,79.81,78,83,9.42,AO,1607530078
4,4,delta del tigre,5.50,6.00,83.35,75,100,5.82,NG,1607530078


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [26]:
# Store 'Lat' and 'Lng' into  locations 
cities = Vacation_Cities[["Lat", "Lng"]].astype(float)

weight = Vacation_Cities["Humidity"].astype(float)

In [28]:
#create Heatmap

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(cities, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
# Create Dataframe based on critera

ideal_city_df = Vacation_Cities.copy()
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp'] > 70]
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp'] < 80]  
ideal_city_df = ideal_city_df[ideal_city_df['Wind Speed'] < 10]
ideal_city_df = ideal_city_df[ideal_city_df['Cloudiness'] == 0]
ideal_city_df = ideal_city_df.dropna()
ideal_city_df = ideal_city_df.reset_index()
ideal_city_df

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,121,121,pisco,-13.70,-76.22,71.01,77,0,6.93,PE,1607529881
1,127,127,casa grande,13.34,-85.37,74.50,58,0,6.38,NI,1607530097
2,142,142,chuy,-33.70,-53.46,76.59,52,0,6.80,UY,1607530099
3,154,154,myingyan,21.47,95.38,70.79,64,0,3.11,MM,1607530101
4,170,170,dawei,14.08,98.20,70.47,81,0,3.44,MM,1607530103
5,179,179,mudon,16.26,97.72,70.83,82,0,3.22,MM,1607530105
6,302,302,tharad,24.40,71.63,70.59,34,0,3.80,IN,1607530125
7,347,347,yanam,16.73,82.22,73.40,88,0,4.70,IN,1607530132
8,361,361,shilong,23.11,113.84,72.00,60,0,2.24,CN,1607530134
9,377,377,tombouctou,20.00,-3.00,78.55,15,0,9.44,ML,1607530137


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
hotel_df = ideal_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,121,pisco,-13.70,-76.22,71.01,77,0,6.93,PE,1607529881,NaN
1,127,casa grande,13.34,-85.37,74.50,58,0,6.38,NI,1607530097,NaN
2,142,chuy,-33.70,-53.46,76.59,52,0,6.80,UY,1607530099,NaN
3,154,myingyan,21.47,95.38,70.79,64,0,3.11,MM,1607530101,NaN
4,170,dawei,14.08,98.20,70.47,81,0,3.44,MM,1607530103,NaN
5,179,mudon,16.26,97.72,70.83,82,0,3.22,MM,1607530105,NaN
6,302,tharad,24.40,71.63,70.59,34,0,3.80,IN,1607530125,NaN
7,347,yanam,16.73,82.22,73.40,88,0,4.70,IN,1607530132,NaN
8,361,shilong,23.11,113.84,72.00,60,0,2.24,CN,1607530134,NaN
9,377,tombouctou,20.00,-3.00,78.55,15,0,9.44,ML,1607530137,NaN


In [56]:
#Use API to locate hotels and add to data frame

hotels = []
#checker variable
x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Lng'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except to check if a city turns up    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#append dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,121,pisco,-13.70,-76.22,71.01,77,0,6.93,PE,1607529881,Hotel San Isidro Oficial
1,127,casa grande,13.34,-85.37,74.50,58,0,6.38,NI,1607530097,Hotel Waslala
2,142,chuy,-33.70,-53.46,76.59,52,0,6.80,UY,1607530099,Nuevo Hotel Plaza
3,154,myingyan,21.47,95.38,70.79,64,0,3.11,MM,1607530101,Myanmar Economic Bank Myingyan
4,170,dawei,14.08,98.20,70.47,81,0,3.44,MM,1607530103,Hotel Dawei
5,179,mudon,16.26,97.72,70.83,82,0,3.22,MM,1607530105,Motel Mudon
6,302,tharad,24.40,71.63,70.59,34,0,3.80,IN,1607530125,Shree Nandan Couriyar Service
7,347,yanam,16.73,82.22,73.40,88,0,4.70,IN,1607530132,The Regency Hotel
8,361,shilong,23.11,113.84,72.00,60,0,2.24,CN,1607530134,Cinese Hotel Dongguan Shijie
9,377,tombouctou,20.00,-3.00,78.55,15,0,9.44,ML,1607530137,NaN


In [57]:
#Clean up dataframe
hotel_df_clean = hotel_df.dropna()
hotel_df_clean

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,121,pisco,-13.70,-76.22,71.01,77,0,6.93,PE,1607529881,Hotel San Isidro Oficial
1,127,casa grande,13.34,-85.37,74.50,58,0,6.38,NI,1607530097,Hotel Waslala
2,142,chuy,-33.70,-53.46,76.59,52,0,6.80,UY,1607530099,Nuevo Hotel Plaza
3,154,myingyan,21.47,95.38,70.79,64,0,3.11,MM,1607530101,Myanmar Economic Bank Myingyan
4,170,dawei,14.08,98.20,70.47,81,0,3.44,MM,1607530103,Hotel Dawei
5,179,mudon,16.26,97.72,70.83,82,0,3.22,MM,1607530105,Motel Mudon
6,302,tharad,24.40,71.63,70.59,34,0,3.80,IN,1607530125,Shree Nandan Couriyar Service
7,347,yanam,16.73,82.22,73.40,88,0,4.70,IN,1607530132,The Regency Hotel
8,361,shilong,23.11,113.84,72.00,60,0,2.24,CN,1607530134,Cinese Hotel Dongguan Shijie


In [58]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# Add marker layer ontop of heat map

# Add marker layer ontop of heat map and display
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))